## 3 Datasets:
- MNIST
- Spambase
- 20NG

For each, perform 2 classification algorithms:
- L2-reg Logistic Regression
- Decision Trees


allowed to use library for classification. 

allowed to use library to process data in appropriate formats.

You are required to explain/analyze the model trained in terms of features : for each of the 6 runs list the top F=30 features. 

For the Regression these correspond to the highest-absolute-value F coefficients; 

for Decision Tree they are the first F splits. In particular for Decision Tree on 20NG, report performance for two tree sizes ( by depths of the tree, or number of leaves, or number of splits )

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch as torch
from sklearn.datasets import _twenty_newsgroups
from torchvision import datasets, transforms

# MNIST

In [13]:
# Load the MNIST dataset
mnist_dataset = datasets.MNIST(root='data', download=True, transform=transforms.ToTensor())
features = mnist_dataset.data
labels = mnist_dataset.targets

# Normalize the features
features = features / 255.0

In [14]:
# using pytorch to perform logistic regression:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))
    
# Initialize the model, loss function, and optimizer
input_dim = 28 * 28  # MNIST images are 28x28 pixels
output_dim = 10  # 10 classes for digits 0-9
model = LogisticRegression(input_dim, output_dim)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Prepare the data for training
features = features.view(-1, input_dim).float()
labels = labels.long()

# Training loop
num_epochs = 5
batch_size = 64
num_batches = len(features) // batch_size
for epoch in range(num_epochs):
    for i in range(num_batches):
        # Get the batch data
        batch_features = features[i * batch_size:(i + 1) * batch_size]
        batch_labels = labels[i * batch_size:(i + 1) * batch_size]
        
        # Forward pass
        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
with torch.no_grad():
    correct = 0
    total = 0
    for i in range(num_batches):
        batch_features = features[i * batch_size:(i + 1) * batch_size]
        batch_labels = labels[i * batch_size:(i + 1) * batch_size]
        
        outputs = model(batch_features)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()
    accuracy = correct / total
print(f'Accuracy of the model on the MNIST test images: {accuracy * 100:.2f}%')


Epoch [1/5], Loss: 2.0248
Epoch [2/5], Loss: 1.9195
Epoch [3/5], Loss: 1.8643
Epoch [4/5], Loss: 1.8290
Epoch [5/5], Loss: 1.8038
Accuracy of the model on the MNIST test images: 84.15%


In [15]:
# Analyze model in terms of features, d = top 30 features
def analyze_model(model, input_dim):
    weights = model.linear.weight.data.numpy()
    top_features = np.argsort(np.abs(weights), axis=1)[:, -30:]  # Get top 30 features for each class
    return top_features
top_features = analyze_model(model, input_dim)

print("Top 30 features for each class:")
for i in range(output_dim):
    print(f"Class {i}: {top_features[i]}")

Top 30 features for each class:
Class 0: [357 404 432 377 491 460 405 352 489 488 437 490 351 381 353 379 436 463
 461 433 462 464 435 378 406 409 380 407 434 408]
Class 1: [516 347 517 294 521 430 374 433 466 437 403 377 346 354 381 402 409 489
 351 379 490 462 434 322 323 461 295 350 378 406]
Class 2: [343 265 322 569 376 581 568 540 321 659 153 349 375 316 373 350 318 320
 317 291 348 290 656 345 347 372 657 374 346 319]
Class 3: [344 544 491 377 572 486 516 545 485 542 460 291 317 461 262 487 490 514
 546 289 349 350 543 515 462 488 518 290 489 517]
Class 4: [237 399 372 240 598 543 464 570 402 430 400 463 436 184 373 428 401 542
 374 427 183 209 212 569 211 429 238 182 239 210]
Class 5: [488 461 190 382 296 376 381 516 487 328 319 320 297 346 489 299 355 189
 514 356 375 348 354 219 298 325 327 374 326 347]
Class 6: [513 268 572 296 573 272 240 626 213 297 243 244 571 655 514 486 215 299
 271 656 515 542 241 214 657 298 543 242 270 269]
Class 7: [403 379 268 431 181 186 269 458 15

In [16]:
# Using decision tree classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load the MNIST dataset
mnist_dataset_train = datasets.MNIST(root='data', train = True, download=True)

mnist_dataset = datasets.MNIST(root='data', train = False, download=True)
print(mnist_dataset_train.data.shape)
print(mnist_dataset.data.shape)
# Train features, labels
features = mnist_dataset_train.data
labels = mnist_dataset_train.targets

# Test features, labels
test_features = mnist_dataset.data
test_labels = mnist_dataset.targets

# convert to numpy arrays:
features = features.numpy()
labels = labels.numpy()

test_features = test_features.numpy()
test_labels = test_labels.numpy()

# Normalize the features
features = features / 255.0
test_features = test_features / 255.0

# Reshape the features to 2D array
features = features.reshape(features.shape[0], -1)
test_features = test_features.reshape(test_features.shape[0], -1)

# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()

# Train the Decision Tree Classifier
dt_classifier.fit(features, labels)

# Predict on the testing set
dt_predictions = dt_classifier.predict(test_features)

# Calculate accuracy
dt_accuracy = accuracy_score(test_labels, dt_predictions)
print(f'Accuracy of the Decision Tree Classifier on the MNIST dataset: {dt_accuracy * 100:.2f}%')



torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])
Accuracy of the Decision Tree Classifier on the MNIST dataset: 87.76%


# Spambase


In [17]:
spambase = pd.read_csv('spambase.data', header=None, sep=',')
spambase

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


In [18]:
# Split the dataset into features and labels
s_labels = spambase.iloc[:, -1]
s_features = spambase.iloc[:, :-1]


In [19]:
# Normalize the features
s_features = (s_features - s_features.mean()) / s_features.std()

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
s_features_train, s_features_test, s_labels_train, s_labels_test = train_test_split(s_features, s_labels, test_size=0.2, random_state=42)


In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Using logistic regression

# Initialize the Logistic Regression model
logistic_model = LogisticRegression()

# Train the model
logistic_model.fit(s_features_train, s_labels_train)

# Predict on the testing set
logistic_predictions = logistic_model.predict(s_features_test)

# Calculate accuracy
logistic_accuracy = accuracy_score(s_labels_test, logistic_predictions)
print(f'Accuracy of the Logistic Regression model on the Spambase dataset: {logistic_accuracy * 100:.2f}%')

Accuracy of the Logistic Regression model on the Spambase dataset: 91.97%


In [21]:
# Decision tree classifier

# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()

# Train the Decision Tree Classifier
dt_classifier.fit(s_features_train, s_labels_train)

# Predict on the testing set
dt_predictions = dt_classifier.predict(s_features_test)

# Calculate accuracy
dt_accuracy = accuracy_score(s_labels_test, dt_predictions)
print(f'Accuracy of the Decision Tree Classifier on the Spambase dataset: {dt_accuracy * 100:.2f}%')


Accuracy of the Decision Tree Classifier on the Spambase dataset: 91.31%


# 20NG

In [22]:
ng_train = _twenty_newsgroups.fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
ng_train_feat = ng_train.data
ng_train_labels = ng_train.target

ng_test = _twenty_newsgroups.fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))
ng_test_feat = ng_test.data
ng_test_labels = ng_test.target

In [23]:
# TFIDF Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TFIDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Fit and transform the training features
ng_train_features = tfidf_vectorizer.fit_transform(ng_train_feat)

# Transform the testing features
ng_test_features = tfidf_vectorizer.transform(ng_test_feat)


In [24]:
# Using logistic regression

# Initialize the Logistic Regression model
logistic_model = LogisticRegression()

# Train the model
logistic_model.fit(ng_train_features, ng_train_labels)

# Predict on the testing set
logistic_predictions = logistic_model.predict(ng_test_features)

# Calculate accuracy
logistic_accuracy = accuracy_score(ng_test_labels, logistic_predictions)
print(f'Accuracy of the Logistic Regression model on the 20 Newsgroups dataset: {logistic_accuracy * 100:.2f}%')

Accuracy of the Logistic Regression model on the 20 Newsgroups dataset: 64.83%


In [25]:
# Decision tree classifier

# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()

# Train the Decision Tree Classifier
dt_classifier.fit(ng_train_features, ng_train_labels)

# Predict on the testing set
dt_predictions = dt_classifier.predict(ng_test_features)

# Calculate accuracy
dt_accuracy = accuracy_score(ng_test_labels, dt_predictions)
print(f'Accuracy of the Decision Tree Classifier on the 20 Newsgroups dataset: {dt_accuracy * 100:.2f}%')

# get the top 30 features
def analyze_model(model, input_dim):
    weights = model.coef_
    top_features = np.argsort(np.abs(weights), axis=1)[:, -30:]  # Get top 30 features for each class
    return top_features
top_features = analyze_model(logistic_model, ng_train_features.shape[1])
print("Top 30 features for each class:")

for i in range(len(ng_train.target_names)):
    print(f"Class {i}: {top_features[i]}")

# unpack features thru the tfidf vectorizer
feature_names = tfidf_vectorizer.get_feature_names_out()
top_features = np.array(top_features)
top_features_names = []
for i in range(len(ng_train.target_names)):
    top_features_names.append([feature_names[j] for j in top_features[i]])
print("Top 30 features for each class:")

for i in range(len(ng_train.target_names)):
    print(f"Class {i}: {top_features_names[i]}")


Accuracy of the Decision Tree Classifier on the 20 Newsgroups dataset: 43.02%
Top 30 features for each class:
Class 0: [ 813 4770 2443 4009  975 3639 2563  701 1384 4489 2645 2971 4232  535
 2528 3958 3771 3130 4481 2972 3597  589  724 2440  590 2439  766 3769
 2047  588]
Class 1: [4971 2947 2185 2716 4407 1068 2652 1958 4194 3249 1061 4768 3539 1130
 4246 4766 1857 4777 3418  423 3424 1313  466 4551 1925 1859 2296  176
 2295 2075]
Class 2: [1908 1458  961 2769 2868 4762 3005 4708  336 2992  951  901 2360 1544
 2897 1958 4716 3114 3098 1909 1564 3875 3517 1859 4890 1563 1003  635
 1857 4893]
Class 3: [  58 2868 1565 1544 1261 2177 1056 2964  902 2154  763 2264 3220 1500
 2434 2436 2980 4799 1988  736 3431  842  198 1206 1561 2270 2963  901
 3308 4005]
Class 4: [2863 2142 4065 2760 3311 2950  862 1939 4174    6 2406 2762 1047 4143
 2288 4005 4812 2963 3517 3115 1578 4144 2602 3453 1561  939 3621 4008
  499 2758]
Class 5: [3541 4708 3415 4500 1706 4214 1070 2646 3378 3927 4957 4716 4076 

## Q2: PCA Using library (sklearn compatible)

A) For MNIST dataset, run a PCA-library to get data on D=5 features. Rerun the classification tasks from PB1, compare testing performance with the one from PB1. Then repeat this exercise for D=20

B) Run PCA library on Spambase and repeat one of the classification algorithms. What is the smallest D (number of PCA dimensions) you need to get a comparable test result?

In [26]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from torchvision import datasets, transforms
from pca import pca

In [35]:
# Load the MNIST dataset
mnist_dataset = datasets.MNIST(root='data', download=True, transform=transforms.ToTensor())
features = mnist_dataset.data
labels = mnist_dataset.targets

# Normalize the features
features = features / 255.0

features.shape


torch.Size([60000, 28, 28])

In [36]:
# a) use PCA to reduce the dimensionality of the data to d = 5:
pca = PCA(n_components=5)
features_pca = pca.fit_transform(features.view(-1, 28 * 28).numpy())

print(f'Explained variance ratio: {pca.explained_variance_ratio_}')

Explained variance ratio: [0.09704393 0.07096103 0.06169181 0.05389257 0.04867888]


In [29]:
# data now looks like:
print(features_pca.shape)

(60000, 5)


In [37]:
# Perform training, testing

# Split the dataset into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(features_pca, labels, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(features_train, labels_train)

# Predict on the testing set
predictions = model.predict(features_test)

# Calculate accuracy
accuracy = accuracy_score(labels_test, predictions)
print(f'Accuracy of the Logistic Regression model on the MNIST dataset after PCA d = 5: {accuracy * 100:.2f}%')


Accuracy of the Logistic Regression model on the MNIST dataset after PCA d = 5: 66.95%


In [38]:
# d = 20
pca = PCA(n_components=20)
features_pca = pca.fit_transform(features.view(-1, 28 * 28).numpy())

# Perform training, testing

# Split the dataset into training and testing sets

features_train, features_test, labels_train, labels_test = train_test_split(features_pca, labels, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(features_train, labels_train)

# Predict on the testing set
predictions = model.predict(features_test)

# Calculate accuracy
accuracy = accuracy_score(labels_test, predictions)
print(f'Accuracy of the Logistic Regression model on the MNIST dataset after PCA d = 20: {accuracy * 100:.2f}%')


Accuracy of the Logistic Regression model on the MNIST dataset after PCA d = 20: 87.51%


In [53]:
# B) Run PCA library on Spambase and repeat one of the classification algorithms. What is the smallest D (number of PCA dimensions) you need to get a comparable test result?

# Load the Spambase dataset
spambase = pd.read_csv('spambase.data', header=None, sep=',')
s_labels = spambase.iloc[:, -1]
s_features = spambase.iloc[:, :-1]

# Normalize the features
s_features = (s_features - s_features.mean()) / s_features.std()

# Perform PCA
pca = PCA(n_components=1)
s_features_pca = pca.fit_transform(s_features)

# Split the dataset into training and testing sets
s_features_train, s_features_test, s_labels_train, s_labels_test = train_test_split(s_features_pca, s_labels, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(s_features_train, s_labels_train)

# Predict on the testing set
predictions = model.predict(s_features_test)

# Calculate accuracy
accuracy = accuracy_score(s_labels_test, predictions)
print(f'Accuracy of the Logistic Regression model on the Spambase dataset after PCA d = 1: {accuracy * 100:.2f}%')

# what is this feature?
print(f'Explained variance ratio: {pca.explained_variance_ratio_}')


Accuracy of the Logistic Regression model on the Spambase dataset after PCA d = 1: 84.04%
Explained variance ratio: [0.11564794]


## PCA Self Implementation

In [54]:
# load mnist dataset
mnist_dataset = datasets.MNIST(root='data', download=True, transform=transforms.ToTensor())
features = mnist_dataset.data
labels = mnist_dataset.targets

# Normalize the features
features = features / 255.0



In [56]:
# get covariance matrix
cov_matrix = np.cov(features.view(-1, 28 * 28).numpy().T)

# get eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

# sort the eigenvalues in descending order
sorted_indices = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[:, sorted_indices]

# get explained variance ratio
explained_variance_ratio = eigenvalues / eigenvalues.sum()
print(f'Explained variance ratio: {explained_variance_ratio[:5]}')

# get the top 5 eigenvectors
top_eigenvectors = eigenvectors[:, :5]


Explained variance ratio: [0.09704664+0.j 0.07095924+0.j 0.06169089+0.j 0.05389419+0.j
 0.04868797+0.j]


In [57]:
# Perform training, testing

# Split the dataset into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(features.view(-1, 28 * 28).numpy().dot(top_eigenvectors), labels, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(features_train, labels_train)

# Predict on the testing set
predictions = model.predict(features_test)

# Calculate accuracy
accuracy = accuracy_score(labels_test, predictions)
print(f'Accuracy of the Logistic Regression model on the MNIST dataset after PCA d = 5: {accuracy * 100:.2f}%')



ValueError: Complex data not supported
[[-3.87895452+0.j -0.62961559+0.j -1.24062249+0.j -4.07137788+0.j
  -2.75597119+0.j]
 [-4.9482789 +0.j -0.10051405+0.j  1.82497409+0.j -2.08015664+0.j
   0.3038148 +0.j]
 [ 0.62456308+0.j -2.89567003+0.j  2.16542742+0.j -1.69270534+0.j
  -3.43336784+0.j]
 ...
 [-7.74606021+0.j -1.70969164+0.j  3.93920483+0.j -2.92376408+0.j
  -1.65427458+0.j]
 [-6.05845876+0.j -4.26704566+0.j  4.0789865 +0.j -2.39895482+0.j
  -2.85616671+0.j]
 [-2.21837206+0.j  1.39129954+0.j  1.7959887 +0.j -0.60472536+0.j
  -4.35491592+0.j]]
